<a href="https://colab.research.google.com/github/AshvinVignesh/Emotion-Detection-By-Speech/blob/main/Emotion_detection_by_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa as lr
import os, glob, pickle
from glob import glob
import soundfile
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
os.listdir(path='/content/gdrive/My Drive/audio')
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = '/content/gdrive/My Drive/audio'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)

1380

In [ ]:
#Now Cleaning Step is Performed where:
#DOWN SAMPLING OF AUDIO FILES IS DONE  AND PUT MASK OVER IT AND DIRECT INTO CLEAN FOLDER
#MASK IS TO REMOVE UNNECESSARY EMPTY VOIVES AROUND THE MAIN AUDIO VOICE 
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [ ]:
#Feature Extraction of Audio Files Function 
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [ ]:
#Emotions in the RAVDESS dataset to be classified Audio Files based on . 
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#These are the emotions User wants to observe more :
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
from glob import glob
import os
import glob
def load_data(test_size=0.15):
    x,y=[],[]
    answer = 0
    for file in glob.glob(r'/content/gdrive/My Drive/cleanAudio//*.wav'):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            answer += 1
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append([emotion,file_name])
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
import librosa
import numpy as np
x_train,x_test,y_trai,y_tes=load_data(test_size=0.15)
print(np.shape(x_train),np.shape(x_test), np.shape(y_trai),np.shape(y_tes))
y_test_map = np.array(y_tes).T
y_test = y_test_map[0]
test_filename = y_test_map[1]
y_train_map = np.array(y_trai).T
y_train = y_train_map[0]
train_filename = y_train_map[1]
print(np.shape(y_train),np.shape(y_test))
print(*test_filename,sep="\n")

(625, 180) (111, 180) (625, 2) (111, 2)
(625,) (111,)
03-01-03-02-01-01-01.wav
03-01-07-02-02-01-08.wav
03-01-06-01-01-02-09.wav
03-01-06-01-02-02-19.wav
03-01-02-01-01-02-02.wav
03-01-03-02-01-02-20.wav
03-01-06-02-02-01-05.wav
03-01-06-01-01-02-02.wav
03-01-06-02-02-01-04.wav
03-01-03-02-01-01-11.wav
03-01-07-01-02-02-07.wav
03-01-03-01-01-02-11.wav
03-01-07-01-02-02-06.wav
03-01-03-02-01-01-06.wav
03-01-02-01-02-01-01.wav
03-01-02-01-02-02-21.wav
03-01-02-01-01-02-15.wav
03-01-07-01-01-01-17.wav
03-01-02-01-01-02-13.wav
03-01-03-02-02-01-22.wav
03-01-02-02-02-02-02.wav
03-01-02-01-01-01-12.wav
03-01-07-01-02-02-15.wav
03-01-07-02-01-02-24.wav
03-01-06-01-02-01-18.wav
03-01-02-01-01-02-18.wav
03-01-03-02-02-02-04.wav
03-01-06-01-01-02-04.wav
03-01-07-02-02-01-22.wav
03-01-03-02-02-02-24.wav
03-01-02-02-01-02-24.wav
03-01-07-01-02-02-22.wav
03-01-02-02-01-01-10.wav
03-01-06-01-01-02-19.wav
03-01-02-01-01-02-12.wav
03-01-07-01-02-01-07.wav
03-01-06-01-01-02-18.wav
03-01-03-02-02-01-01.

In [ ]:
#Get the shape of the training and testing datasets
# print((x_train.shape[0], x_test.shape[0]))
print((x_train[0], x_test[0]))
print((x_train.shape[0], x_test.shape[0]))
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

(array([-2.18390244e+02,  3.74150124e+01, -6.37748146e+01,  1.59183550e+01,
       -3.00963478e+01, -2.30383816e+01, -3.12656689e+01, -3.57604637e+01,
       -6.94720650e+00, -1.85371838e+01, -2.29447899e+01,  4.58464050e+00,
       -1.06219997e+01,  1.83856697e+01, -1.74620962e+00,  1.41900511e+01,
        1.45657692e+01,  6.09453678e+00,  2.49503469e+00,  1.11287193e+01,
       -1.37202531e-01,  2.49402881e+00, -1.11175370e+00,  8.60968173e-01,
       -5.15002680e+00,  1.56028700e+00, -6.00139713e+00,  2.58057666e+00,
       -3.18782139e+00, -1.18974721e+00, -4.39024448e+00,  4.16844893e+00,
       -4.86300278e+00,  4.64808512e+00, -3.14665651e+00,  2.77286553e+00,
       -2.12462330e+00,  3.57387066e+00, -8.32439840e-01, -1.77118778e+00,
        3.53982121e-01,  3.84131014e-01,  3.61878037e-01,  3.81937027e-01,
        4.36897904e-01,  4.68189120e-01,  4.98520851e-01,  5.58137596e-01,
        6.51815116e-01,  6.65794551e-01,  5.76719999e-01,  3.96059960e-01,
        1.02497656e-02, 

In [ ]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#predicting :
y_pred=model.predict(x_test)
y_pred

array(['happy', 'disgust', 'fearful', 'fearful', 'calm', 'happy',
       'fearful', 'fearful', 'fearful', 'happy', 'disgust', 'calm',
       'calm', 'fearful', 'calm', 'calm', 'calm', 'disgust', 'calm',
       'happy', 'calm', 'calm', 'disgust', 'disgust', 'fearful', 'calm',
       'happy', 'happy', 'disgust', 'happy', 'calm', 'disgust', 'calm',
       'fearful', 'calm', 'calm', 'fearful', 'happy', 'calm', 'disgust',
       'disgust', 'disgust', 'disgust', 'calm', 'fearful', 'disgust',
       'fearful', 'disgust', 'happy', 'happy', 'calm', 'calm', 'calm',
       'disgust', 'disgust', 'fearful', 'fearful', 'calm', 'happy',
       'disgust', 'happy', 'disgust', 'fearful', 'fearful', 'fearful',
       'happy', 'calm', 'fearful', 'calm', 'fearful', 'calm', 'happy',
       'fearful', 'calm', 'disgust', 'fearful', 'happy', 'happy', 'calm',
       'calm', 'fearful', 'calm', 'fearful', 'fearful', 'happy',
       'disgust', 'happy', 'disgust', 'disgust', 'disgust', 'happy',
       'disgust', 'f

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 85.59%


In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,disgust,disgust
2,fearful,fearful
3,fearful,fearful
4,calm,calm
5,happy,happy
6,fearful,fearful
7,fearful,fearful
8,fearful,fearful
9,happy,happy


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/03-01-08-02-02-01-17.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['disgust'], dtype='<U7')

In [ ]:
#to check